In [40]:
import nglview as nv
import parmed as pmd

from openmm import app as mm
import openmm  as omm
 
import parmed.unit as u

In [62]:
but = mm.PDBFile('..\Open_MM\Butanol-1\Structure.pdb')
but_ff = mm.ForceField('..\Open_MM\Butanol-1\FF.xml')
but_sys = but_ff.createSystem(
    but.topology,
    nonbondedMethod=mm.NoCutoff,
)


In [53]:
but_sys

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x00000213CDFE9DD0> >

In [54]:
bu3 = pmd.openmm.load_topology(butanol.topology,but_sys)

In [60]:
bu3.coordinates

In [22]:
butanol2= pmd.load_file('..\Open_MM\Butanol-1\Structure.pdb',structure=False)
butanol2

<Structure 15 atoms; 1 residues; 14 bonds; NOT parameterized>